In [169]:
import MetaTrader5 as mt5
from IPython.display import display
from datetime import datetime
import plotly.express as px

from projects.atj_trading_legacy.backtester import get_ohlc_history, create_price_fig
from config import mt5_credentials

In [170]:
mt5.initialize()
mt5.login(mt5_credentials['login'], mt5_credentials['password'], mt5_credentials['server'])

True

In [171]:
symbol = 'TSLA'
timeframe = mt5.TIMEFRAME_D1

start_dt = datetime(2020, 1 ,1)
end_dt = datetime.now()

ohlc = get_ohlc_history(symbol, timeframe, start_dt, end_dt)
display(ohlc)

create_price_fig(ohlc.tail(100))

,time,open,high,low,close
0,2020-01-02,28.40,28.70,28.11,28.67
1,2020-01-03,29.66,30.26,29.39,29.52
2,2020-01-06,29.67,30.06,29.41,30.05
3,2020-01-07,30.61,31.43,30.21,31.21
4,2020-01-08,31.64,33.22,31.21,32.77
...,...,...,...,...,...
1161,2024-08-14,205.91,205.96,198.74,201.02
1162,2024-08-15,208.61,215.85,208.33,214.01
1163,2024-08-16,214.34,219.76,214.09,216.61
1164,2024-08-19,215.46,222.95,214.08,222.69


In [172]:
ohlc2 = ohlc.tail(100).copy()

fig_ohlc = create_price_fig(ohlc2)
fig_ohlc.show()

In [173]:
# Support Analysis
count_limit = 5

def custom_round(x, base=5):
    return int(base * round(float(x)/base))


ohlc2['low_level'] = ohlc2['low'].apply(custom_round)
print(ohlc2)

ohlc2['count_lows'] = 1

support_df = ohlc2.groupby('low_level', as_index=False)['count_lows'].count()
display(support_df)

fig = px.bar(support_df, x='low_level', y='count_lows')
fig.add_hline(y=count_limit, line_dash='dash')

           time    open    high     low   close  low_level
1066 2024-03-28  179.20  179.23  175.30  175.70        175
1067 2024-04-01  175.90  176.16  170.21  174.04        170
1068 2024-04-02  163.90  167.67  163.42  166.35        165
1069 2024-04-03  165.91  168.79  163.27  168.36        165
1070 2024-04-04  168.60  177.15  168.02  171.73        170
...         ...     ...     ...     ...     ...        ...
1161 2024-08-14  205.91  205.96  198.74  201.02        200
1162 2024-08-15  208.61  215.85  208.33  214.01        210
1163 2024-08-16  214.34  219.76  214.09  216.61        215
1164 2024-08-19  215.46  222.95  214.08  222.69        215
1165 2024-08-20  227.07  227.65  219.55  221.83        220

[100 rows x 6 columns]


,low_level,count_lows
0,140,2
1,145,1
2,150,1
3,155,2
4,160,4
5,165,4
6,170,14
7,175,20
8,180,10
9,185,4


In [174]:
# Resistance Analysis

ohlc2['high_level'] = ohlc2['high'].apply(custom_round)
ohlc2['count_highs'] = 1

resistance_df = ohlc2.groupby('high_level', as_index=False)['count_highs'].count()
display(resistance_df)

fig = px.bar(resistance_df, x='high_level', y='count_highs')
fig.add_hline(y=count_limit, line_dash='dash')

,high_level,count_highs
0,145,2
1,150,2
2,160,2
3,170,8
4,175,14
5,180,15
6,185,11
7,190,6
8,200,6
9,205,5


In [175]:
fig_sr = create_price_fig(ohlc2)
fig_sr.update_layout(title='Support Levels')

for i, x in support_df.iterrows():
    if x['count_lows'] >= count_limit:
        fig_sr.add_hline(x['low_level'], line_color='yellow', opacity=0.5)
        
fig_sr

In [176]:
fig_sr = create_price_fig(ohlc2)
fig_sr.update_layout(title='Resistance Levels')

for i, x in resistance_df.iterrows():
    if x['count_highs'] >= count_limit:
        fig_sr.add_hline(x['high_level'], line_color='yellow', opacity=0.5)
        
fig_sr      
        
